In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def get_Prolific_data(sheet):
    print('--------------------------------')
    print(f'Prolific {sheet} sheet')
    print()
    
    if sheet not in ['Trust', 'Readability']:
        raise ValueError('Sheet name not found!')
        
    start_column, end_column = (5, 30) if sheet == 'Readability' else (6, 31)
    
    # read excel and get batchinitial info
    df = pd.read_excel('Prolific Data.xlsx', sheet_name=sheet).iloc[:, start_column:end_column].dropna()
    print(f'Initial data size: {len(df)} rows')
    print('--------------------------------')
    return df

In [3]:
def process_Prolific_data(data):
    # group by question_id
    df = pd.DataFrame(columns=['question_id', 'answer'])
    for i in range(1, 13):
        column1 = f'image_{i}'
        column2 = f'image_{i}_answer'
        to_concat = data[[column1, column2]].rename(columns={column1: 'question_id', column2: 'answer'}, inplace=False)
        df = pd.concat([df, to_concat])
    print(f'Size after processing data: {len(df)} = 12 questions * {len(data)} records.')
    print()
    return df

In [4]:
def is_processed_batch(file, source):
    params = dict()
    processed = False
    if os.path.isfile(file):
        sources = list(pd.read_csv(file, usecols=['source']).source.unique())
        if source in sources:
            processed = True
        else:
            params['mode'] = 'a'
            params['header'] = None
    return (processed, params)

In [5]:
def update_ML_csv(Readability, Trust, batch_number):
    source = f'prolific-batch{batch_number}'
    
    processedR, paramsR = is_processed_batch('MLR.csv', source)
    processedT, paramsT = is_processed_batch('MLT.csv', source)
    
    if not processedR:
        R = Readability.copy(deep=True)
        R['source'] = source
        R.to_csv('MLR.csv', index=False, **paramsR)
        print(f'Updated MLR.csv with the latest "{source}" !')
    else:
        print(f'Readability {source} already processed!')
    
    if not processedT:
        T = Trust.copy(deep=True)
        T['source'] = source
        T.to_csv('MLT.csv', index=False, **paramsT)
        print(f'Updated MLT.csv with the latest "{source}" !')
    else:
        print(f'Trust {source} already processed!')

In [6]:
Readability = process_Prolific_data(get_Prolific_data('Readability'))
Trust = process_Prolific_data(get_Prolific_data('Trust'))

update_ML_csv(Readability, Trust, 1)

--------------------------------
Prolific Readability sheet

Initial data size: 1634 rows
--------------------------------
Size after processing data: 19608 = 12 questions * 1634 records.

--------------------------------
Prolific Trust sheet

Initial data size: 1626 rows
--------------------------------
Size after processing data: 19512 = 12 questions * 1626 records.

Updated MLR.csv with the latest "prolific-batch1" !
Updated MLT.csv with the latest "prolific-batch1" !
